### BIAI PROJECT


AUTHORS: Konrad Sygut, Dominik Baryś

In [1]:
import os
import glob
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pickle
from mpl_toolkits.axes_grid1 import ImageGrid
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn import metrics
import matplotlib.pyplot as plt

### Getting list of images (excluding some that do not open)

In [2]:
PATH_TO_IMAGES = '/kaggle/input/cats-and-dogs-breeds-classification-oxford-dataset/images/images'
PATH_TO_TRIMAPS = '/kaggle/input/cats-and-dogs-breeds-classification-oxford-dataset/annotations/annotations/trimaps'
# setting image pixel side size
SIZEOF_IMAGE = 299

bad = {'Abyssinian_34.jpg', 'Egyptian_Mau_145.jpg', 'Egyptian_Mau_139.jpg', 'Egyptian_Mau_191.jpg', 'Egyptian_Mau_177.jpg', 'Egyptian_Mau_167.jpg'}

all_imgs = [i for i in os.listdir(PATH_TO_IMAGES) if i.rsplit('.',1)[1] == 'jpg' and i not in bad]
all_trimaps = [i for i in os.listdir(PATH_TO_TRIMAPS) if i.rsplit('.',1)[1] == 'png']

print('There are ' + str(len(all_imgs)) + ' images.')


There are 7384 images.


In [3]:
# getting info from 'list.txt' file
l = open('/kaggle/input/cats-and-dogs-breeds-classification-oxford-dataset/annotations/annotations/list.txt', 'r')
get_breed = lambda pic : pic.rsplit('_',1)[0].lower()
get_species = lambda num : 'cat' if num==1 else 'dog'

info_by_id = {}
info_by_breed = {}

# taking note of the names and ids of the breeds
for line in l:
  if line[0] == '#':
    continue
  line = line.strip().split(' ')
  species = get_species(int(line[2]))
  id = int(line[1])
  breedid = int(line[3])
  name = get_breed(line[0]).lower()
  if name not in info_by_breed:
    info_by_breed[name] = {'breed' : name, 'species' : species, 'globalid': id, 'breedid':breedid, 'count':0}
    info_by_id[id] = info_by_breed[name]

# to count the images we can't trust the file
for p in [get_breed(n) for n in all_imgs]:
  info_by_breed[p]['count']+=1


### To get information about the images from the `list.txt` file
Information is extracted into 2 dictionaries: `info_by_id` and `info_by_breed`

In [ ]:
ids = list(info_by_id.keys())

# X value:
counts = [info_by_id[id]['count'] for id in ids]
x_labels = [info_by_id[i]['breed'] for i in ids]

# Colours & legend:
colours = [ 'green' if info_by_id[id]['species']=='cat' else 'purple' for id in ids]

colours_leg = {'cat': 'green', 'dog':'purple'}
labels = list(colours_leg.keys())
handles = [plt.Rectangle((0,0),1,1, color=colours_leg[label]) for label in colours_leg]

# Plotting:
fig, ax = plt.subplots( figsize= (11,9))
ax.barh(ids, counts, color=colours) # barh instead of bar

# Set ticks & axis labels & legend:
ax.set_yticks(ids) # set_yticks instead of set_xticks
ax.set_yticklabels(x_labels) # set_yticklabels instead of set_xticklabels
plt.legend(handles, labels)
plt.ylabel('Cats and dogs breeds') # swapped places
plt.xlabel('Amount of pictures') # swapped places
plt.title('Division of cats and dogs by breed')

# Set X axis limit:
plt.xlim(0, 225)

plt.savefig('wykres.png', bbox_inches='tight')

plt.show()

nr_cats = sum([ info_by_id[id]['count'] for id in ids if info_by_id[id]['species'] == 'cat' ])
nr_dogs = sum([ info_by_id[id]['count'] for id in ids if info_by_id[id]['species'] == 'dog' ])



### Display a bar chart of the number of images per breed & couting images per species

### _Reading_ all images, resizing and adding to list

In [ ]:
def getXy(imgs=None):
  # function that returns the number correspondent to the breed of   the animal in the image, given the image name
  get_class_no = lambda name : info_by_breed[get_breed(name)]  ['globalid']
  
  # this set was only used in the begining, before knowing which   images were not opening
  # bad = set()
  
  # all image tensors will be stored here after resizing
  training_data = []
  
  for img in all_imgs:
    path = os.path.join(PATH_TO_IMAGES, img)
  
    # this is a trick to make the image be opened in RGB format,   which is not the default
    img_array = cv2.imread(path)[...,::-1] 
  
    # this next block of code, just like the 'bad' set, was   used before finding out "bad" images
    # if img_array is None:
    #   bad.add(img)
    #   continue
  

    # here the images are rezise
    img_array = cv2.resize(img_array, (SIZEOF_IMAGE, SIZEOF_IMAGE))
  
    # get the ID of the image class
    class_no = get_class_no(img)
  
    if imgs is not None and class_no not in imgs:
      imgs[class_no] = path
  
    training_data.append([img_array, class_no])
    
  # data should be in random order to improve performance
  random.shuffle(training_data)
  
  # separating data from list
  training = list(zip(*training_data))
  X = training[0]
  y = training[1]
  
  # transforming X to an np.array and resizing
  X = np.array(X).reshape(-1, SIZEOF_IMAGE, SIZEOF_IMAGE, 3)
  return X, y


### Saving this data to files to make it easier to use it

In [ ]:
def save(obj, fic_name, open_type='wb'):
  pickle_out = open(fic_name, open_type)
  pickle.dump(obj, pickle_out)
  pickle_out.close()

In [ ]:
# this is a dictionary that is going to be used to map the ID to a   path to an image, with the same goal as the list before
imgs = {}

X, y = getXy(imgs=imgs)
save(X, 'X299.pickle')
save(y, 'y299.pickle')
print(X.shape)


### Display a chart containing one image per breed in the data set

In [ ]:
# getting a list of all images we want to show in order
ids = list(imgs.keys())
figs = [cv2.resize(cv2.imread(imgs[i])[...,::-1], (SIZEOF_IMAGE, SIZEOF_IMAGE)) for i in ids]

fig = plt.figure(figsize=(50,50))
grid = ImageGrid(
    fig,
    111,
    nrows_ncols=(6, 6),
    axes_pad=0.5
)

i = 1
for ax, im in zip(grid, figs):
    # putting the correspondent number at the top:
    breed_name = info_by_id[ids[i-1]]['breed']  # Pobranie nazwy rasy dla danego numeru
    ax.set_title(f"{i}: {breed_name}", loc='center', fontsize=32)  # Dodanie nazwy rasy do tytułu
    ax.imshow(im)
    ax.axis('off')
    i+=1

fig.subplots_adjust(top=1.27)
fig.suptitle('Examples of Pet Image per Breed', size='large')
plt.savefig('nazwa_pliku.png', bbox_inches='tight')
plt.show()


### Loading files

In [ ]:
X = pickle.load(open("X299.pickle","rb"))
X = np.array(X)

y = pickle.load(open("y299.pickle","rb"))
y = np.array(y)


### Split data into training and testing data
This split is stratified, which means that the ratios between the numbers of images in each class will be kept equal in the testing set

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

### Function that returns the (complex) model according to some variables

In [ ]:
def getModel(dropout=.25, learning_rate=0.001):
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
  base_model.trainable = False

  inputs = tf.keras.Input(shape=(299, 299, 3))
  x = inputs
  x = tf.keras.applications.inception_v3.preprocess_input(x)
  x = base_model(x, training=False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(256,activation='relu')(x)
  x = tf.keras.layers.Dropout(dropout)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  outputs = tf.keras.layers.Dense(37,activation='softmax')(x)
  model = tf.keras.Model(inputs, outputs)
  model.summary()

  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

  return model

### Function to encode `y` to arrays of 0's and 1's so it checks out with the model we have

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
def onehotencode_func(y):
  integer_encoded = y.reshape(len(y), 1)
  onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
  return onehot_encoded

### Defining values that will be experimented with and setting up k-fold cross validation

In [ ]:
learning_rate_list = [0.01, 0.001]
dropout_values_list = [0.25, 0.35]

# 3-fold cross validation will be used because its computationally easier/faster
kfold = StratifiedKFold(n_splits=3, shuffle=True)

# dictionary where data will be stored
hist = {'learning_rate': {}, 'neurons': {}, 'dropout': {}}

### Functions used later to save data from hyper parameter tuning on files, and to read them as well

In [ ]:
def save_in_file(parameter, dict, filename):
  f = open(filename, 'ab')
  pickle.dump({parameter : dict[parameter]}, f)
  f.close()

# returns a list
def read_file(filename):
  objs = [] 
  f = open(filename, 'rb')
  while 1:
      try:
          objs.append(pickle.load(f))
      except EOFError:
          break
  f.close()
  return objs

### Function where information from k-fold cross validation will be averaged, stored and returned

In [ ]:
# the 'model_func' parameter is a lambda function
def test_params(lr, model_func):
  dic = {}
  i = 0.0

  # splitting data into the folds
  folds = kfold.split(x_train, y_train)
  for train_index, val_index in folds:

    # getting the model with the desired parameters
    model = model_func(lr)

    x_train_kf, x_val_kf =  x_train[train_index], x_train[val_index]
    y_train_kf, y_val_kf = onehotencode_func(y_train[train_index]), onehotencode_func(y_train[val_index])

    # training the model with data from the train data folds
    historytemp = model.fit(x_train_kf, y_train_kf, batch_size=32, epochs=15, validation_data=(x_val_kf, y_val_kf))

    del model

    if dic == {}:
      # if dictionary is empty, values will be put there
      dic['train_acc'] = np.array(historytemp.history['accuracy'])
      dic['train_loss'] = np.array(historytemp.history['loss'])
      dic['val_acc'] = np.array(historytemp.history['val_accuracy'])
      dic['val_loss'] = np.array(historytemp.history['val_loss'])
    else:
      # if dictionary is not empty, values will be added element wise
      dic['train_acc'] += np.array(historytemp.history['accuracy'])
      dic['train_loss'] += np.array(historytemp.history['loss'])
      dic['val_acc'] += np.array(historytemp.history['val_accuracy'])
      dic['val_loss'] += np.array(historytemp.history['val_loss'])
    
    i+=1

  for k in dic:
    # each number in each array in the dictionary will be divided by the number of iterations, producing the mean of all the values read
    dic[k] /= i

  return dic

### Setting up the experiences

In [ ]:
# changing learning rate:
lr_model_func = lambda x : getModel(learning_rate=x)
for lr in learning_rate_list:
  hist['learning_rate'][lr] = test_params(lr, lr_model_func)

save_in_file('learning_rate', hist, 'data299-simple.pickle')


# chaning dropout value:
drop_model_func = lambda x : getModel(dropout=x)
for d in dropout_values_list:
  hist['dropout'][d] = test_params(d, drop_model_func)

save_in_file('dropout', hist, 'data299-simple.pickle')


print(read_file('data299-simple.pickle'))

### Displaying a representation of the neural network architecture

In [ ]:
m = getModel()

### Displaying the confusion matrix and metrics table
Through our analysis, we discovered that the best parameters were:
* **learning rate** = 0.001
* **dropout value** = 0.35

get a new model and train it with all the training data available:

model = getModel(learning_rate=0.001, dropout=0.35)
model.fit(x_train, onehotencode_func(y_train), batch_size=32, epochs=15)

Getting the predictions of the test data and transforming it to a number (selecting the index of the maximum value and summing one):

In [ ]:
y_pred = model.predict(x_test)
y_pred2 = [ np.argmax(i)+1 for i in y_pred]

In [ ]:
# list of labels in order gotten from the previous notebook
labels = ['abyssinian', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'bengal', 'birman', 'bombay', 'boxer', 'british_shorthair', 'chihuahua', 'egyptian_mau', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees', 'havanese', 'japanese_chin', 'keeshond', 'leonberger', 'maine_coon', 'miniature_pinscher', 'newfoundland', 'persian', 'pomeranian', 'pug', 'ragdoll', 'russian_blue', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'siamese', 'sphynx', 'staffordshire_bull_terrier', 'wheaten_terrier', 'yorkshire_terrier']

In [ ]:
import seaborn as sns
import pandas as pd

# Tworzenie macierzy pomyłek
cm = metrics.confusion_matrix(y_test, y_pred2)

cm_df = pd.DataFrame(cm, index=labels, columns=labels)

plt.figure(figsize=(12, 12))


sns.heatmap(cm_df,fmt='g', cmap='Blues', cbar=False)

plt.title('Confusion Matrix', size='xx-large')
plt.ylabel('Actual Breed')
plt.xlabel('Predicted Breed')

plt.xticks(rotation=90)  # Obróć etykiety osi x o 90 stopni
plt.savefig('wykres.png', bbox_inches='tight')
plt.show()


### Fine-tuning transfer learning model

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.RMSprop(lr=0.001/10),
              metrics=['accuracy'])

### Function to read pickle files to a list

In [ ]:
def read_file(f):
  objs = [] 
  f = open(f, 'rb')
  while 1:
      try:
          objs.append(pickle.load(f))
      except EOFError:
          break
  f.close()
  return objs

### Reading the files with historic information about the hyper parameter tuning
The name of the file passed to the `read_file` function should be changed to the correct name of the file

In [ ]:
l = read_file('data299-simple.pickle')
lr = l[-2]['learning_rate']
d = l[-1]['dropout']


# dictionary that maps the keys of the dictionaries above to the "normal" name of the metric
metric2name = {'train_acc' : 'train accuracy', 'train_loss' : 'train loss', 'val_acc': 'validation accuracy', 'val_loss': 'validation loss'}

### Displaying charts
Evaluating different learning rates:

In [ ]:
fig, axs = plt.subplots(2, figsize=(15, 15))

metric_colors = {
    "train accuracy": "orange",
    "validation accuracy": "black",
    "train loss": "darkgreen",
    "validation loss": "darkblue"
}

i = 0
for idx, val in enumerate(d):
    for metric in d[val]:
        axs[i].plot(d[val][metric], color=metric_colors[metric2name[metric]], linestyle='solid', label=metric2name[metric])
    axs[i].set_title('Drop rate: ' + str(val), fontsize=16)
    axs[i].set_xlabel('Epoch Count', fontsize=14)
    axs[i].set_ylabel('Metric Value', fontsize=14)
    axs[i].legend(loc='upper right')
    axs[i].set_facecolor('lightgrey')
    axs[i].grid(True)
    axs[i].set_ylim([0, 1.2])
    i += 1

fig.suptitle('Metric Changes over Learning Rates', size='xx-large', y=1.05)
fig.tight_layout()
plt.savefig('droprate-complex.png', bbox_inches='tight')
plt.show()

Evaluating different dropout values:

In [ ]:
fig, axs = plt.subplots(2, figsize=(15, 15))

metric_colors = {
    "train accuracy": "orange",
    "validation accuracy": "black",
    "train loss": "darkgreen",
    "validation loss": "darkblue"
}

i = 0
for idx, val in enumerate(lr):
    for metric in lr[val]:
        # Przypisujemy kolor na podstawie nazwy metryki
        axs[i].plot(lr[val][metric], color=metric_colors[metric2name[metric]], linestyle='solid', label=metric2name[metric])
    axs[i].set_title('Learning Rate: ' + str(val), fontsize=16)
    axs[i].set_xlabel('Epoch Count', fontsize=14)
    axs[i].set_ylabel('Metric Value', fontsize=14)
    axs[i].legend(loc='upper right')
    axs[i].set_facecolor('lightgrey')
    axs[i].grid(True)

    axs[i].set_ylim([0, 1.2])
    i += 1

fig.suptitle('Metric Changes over Learning Rates', size='xx-large', y=1.05)
fig.tight_layout()
plt.savefig('learningrate-complex.png', bbox_inches='tight')
plt.show()